## **Transformer**
- 순서를 고려할 수 있는 Attention 기반의 딥러닝 구조로, RNN 없이도 문장처럼 순차적인 데이터를 효과적으로 처리할 수 있게 해주는 모델이다.
- Transformer는  encoder와 decoder에서 RNN을 제거하고 **Attention 모듈로 변경하여 병렬 처리가 가능**하도록 했다.

| 기존 방식 (RNN, LSTM) | Transformer       |
| ----------------- | ----------------- |
| 순차적으로 처리 (느림)     | 병렬 처리 가능 (빠름)     |
| 긴 문장 처리 어려움       | 멀리 떨어진 단어도 잘 연결함  |
| 구조가 복잡            | Attention 중심으로 간결 |


- Transformer는 크게 Encoder와 Decoder 두 부분으로 구성됩니다.
    - (Encoder): 입력 시퀀스를 처리해 정보(문맥)를 추출.

    - (Decoder): 인코더의 정보를 기반으로 출력 시퀀스를 생성
- 각각은 여러 개의 **층(layer)**으로 구성되며, 각 층은 self-attention과 feed forward 신경망으로 구성

- BERT: Encoder만 사용 (이해 tasks, 예: 문장 분류)

- GPT: Decoder만 사용 (생성 tasks, 예: 문장 생성)

- Encoder와 Decoder가 **self-attention 모듈**로 구성된다. 그래서 RNN을 쓰지 않고 입력 sequence와 출력 sequence를 처리한다.

![img](figures/transformer_architecture1.png)

\[**Encoder와 Decoder의 구조**\]

1. Input Embedding + Positional Encoding
    - Transformer는 순서를 모르는 구조이기 때문에, 단어 임베딩 + 위치 정보(Positional Encoding)를 더해줍니다.
2. Multi-Head Self-Attention
    - 한 번의 attention만으로는 다양한 관점을 반영하기 어려워서 여러 개의 head로 attention을 계산 후 결합합니다.
        - Self-Attention: 입력 시퀀스 내 단어들끼리 서로 얼마나 중요한지 계산합니다.
        - 각 단어는 Query, Key, Value로 변환되고 아래 수식을 따라 Attention Score가 계산됩니다.
3. Feed Forward Network (FFN)
    - 각 위치에서 독립적으로 작동하는 작은 신경망입니다.
4. Add & Norm (Residual + LayerNorm)
    - Residual connection을 통해 정보 손실을 막고, Layer Normalization을 통해 학습 안정성을 높입니다.

### 디코더는 인코더와 유사하지만:
- Masked self-attention을 사용해 미래 토큰을 보지 못하게 합니다.

- 인코더의 출력도 encoder-decoder attention을 통해 활용.

## Self Attention
- Attention을 자기 자신한테 적용한다는 의미
    - seq2seq 의 attention은 decoder의 sequence가 encoder의 hidden state에 attention을 취한다. (디코더가 인코더의 어디에 집중할 지 찾는다.)
    - self attention은 attention을 자기 자신에게 취한다. (같은 문장 내에서 단어들이 서로 어떤 관계를 가지는지 찾는다.)
- Self-attention은 하나의 단어가 자신이 속한 문장의 다른 단어들과 어떤 연관성을 가지는지 찾는다.

![img](figures/transformer_self-attention.png)

- 예를 들어 "The animal didn't cross the street because it was too tired." 라는 문서에 있는  `it`을 encoding 한려고 한다. 
- `it`을 문맥에 맞게 embedding하기 위해서는 이 문서안에서 어떤 의미로 쓰였는지 알아야 한다. 
- 그것을 알기 위해서는 `it` 이 가리키는(관련있는) 것이 어떤 것인지 알아야 한다. 그것을 다른 문서에서 찾는 것이 아니라 `it` 있는(대상 단어가 있는) 문서에서 다른 단어들과의 연관성에서 찾는다.
- **의미를 파악하려는 문서와 그 의미를 찾는 문서가 동일하기 때문에 self-attention**이라고 한다.

### Self Attention 과정
- 자기 자신을 구성하는 부분끼리 연관성을 찾고자 할 때 사용하는 attention 방법.
  
![self-attention](figures/transformer_self_attention.png)

- **수식**
\begin{align}
&\text{Attention(Q,K,V)} = \text{softmax}(\cfrac{QK^T}{\sqrt{d_k})})V \\
&\small \text{Q: Query, K: Key, V: Value,}\;d_{k}: \text{embedding 차원}
\end{align}

1. Query, Key, Value 생성
    - **Query**: Embedding vector를 구할 대상 (ex: it) - 알고싶은 정보
    - **Key**: Query와 연관성있는 단어들을 계산할 대상 (ex: 위 그림의 왼쪽 문서 토큰들) - 각 단어 특징
    - **Value**: Query와 Key를 이용해 찾은 attention weight를 적용해 Attention value를 찾을 대상. - 각 단어 정보
    - Self attention은 Query, Key, Value 모두 입력 Sequence(X)로 부터 생성한다.
       - X에 각각 다른 Weight를 내적하여 만든다.
         - $Query=X\cdot W_q$ 
         - $Key=X\cdot W_k$ 
         -  $Value=X\cdot W_v$

![img](figures/transformer_query_key_value.png)

2. **Attention Score**
    - Query 와 Key를 내적(dot product)하여 유사도를 계산한다. embedding vector의 차원의 제곱근으로 나눠서 정규화한다.
    - Scaled Dot Product Attention
   $$
   \text{Attention Score} = \cfrac{Q\cdot K^T}{\sqrt{d_k}}
   $$
![img](figures/transformer_query_key_matmulpng.png)

3. **Attention Weight(Distribution)**
   - 위에서 계산된 Attention score에 softmax를 적용해 0 ~ 1 사이 비율로 바꾼다. -> V에 가중 평균 적용
     
   $$
    \text{Attention Weight} = softmax(\text{Attention Score})
   $$

4. **Attention value**
   - 최종 attention 연산의 결과로 **Attention weight를 Value에 내적**해서 Attention Value를 만든다.
   - Attention에서 Attention value값이 **입력 sequence의 context vector**가 된다.
   $$
   \text{Attention Value} = \text{Attention Weight}\cdot\text{Value}
   $$ 
![img](figures/transformer_attention_value.png)

## Multi-Head attention
- Multi-Head Attention 은 입력 데이터의 Attention Value를 계산하는 Query, Key, Value의 weight들을 하나가 아니라 여러개로 작게 나눠서 계산한다.
- 이렇게 나눈 것을 head라고 하고 여러개를 만들어 사용하므로 multi-head attention이라고 한다.
- **분할 과정**
    - 입력 Embedding Vector가 512차원이고 Head를 8개 사용한다고 할 경우
    - Head는 64차원(512/8) 의 서로 다른 weight를 가진 Query, Key, Value를 8개 생성한다. (Single Head는 512차원 한개의 Query, Key, Value 생성)
        - 입력 차원이 head 수로 나누어떨어지는 경우에만 head 분할이 가능하다.  `입력차원 % head수 = 0` 이어야 한다.
    - 각 head는 독립적으로 Attention 연산을 수행한다. (**병렬 처리**)
    - 모든 head의 출력을 concat한 후, 최종적으로 출력 차원을 원래 차원으로 매핑하기 위해 추가적인 선형 변환을 수행한다.
- **장점**
    -  동일한 입력 시퀀스에 대해 여러 관점(문법적 관계, 의미적 관계 등)에서의 정보를 동시에 학습할 수 있다.
    -  각 head가 서로 다른 특성을 학습하여 더 풍부한 표현이 가능하다.
    - 여러 head의 연산이 병렬적으로 처리될 수 있어 GPU와 같은 하드웨어에서 연산 속도를 극대화할 수 있다.


![img](figures/transformer_multi-head-attention.png)

## Masked self Attention
- 입력 sequence 가 순서대로 입력되어 처리 되는 경우 i번째 입력 단어의 경우 그 이후의 단어는 모르는 상태이다. 
- Attention은 입력된 모든 토큰을 한번에 처리해버리기 때문에 attention score를 구할 수있게 된다. 이것은 주어진 토큰(i번째)이 미래시점의 입력토큰(i+1 이후 번째)과의 유사도를 계산한 것이 된다. 
- 이 문제를 해결하기 위해 i번째 토큰에 대한 attention score는 i번째 까지의 토큰들과만 계산하도록 한다.
- Attention Score를 Softmax 계산하기 전에 적용한다.
- Attention 계산시 선택적으로 mask를 사용하면 Masked self attention이 된다. (Masked Multi-Head Attention)
![img](figures/transformer_masked_self_attention.png)